In [1]:
import sys
import json
from pathlib import Path


if ".." not in sys.path:
    sys.path.append("..")

from src.data_store import DataStore


## Create store from JSON

In [2]:

_TEST_JSON = "../test_runs/kql_query_db-022-09-23_00_44_55.json"

ds = DataStore(json_path=_TEST_JSON)


## Simple OPs

In [7]:
# queries brings the whole set back as a list
print(ds.queries[3], "\n")

# also get the whole set as JSON or DF
print("\nJSON\n", ds.to_json()[:300], "\n")
ds.to_df().head(3)

KqlQuery(source_path='https://github.com/Azure/Azure-Sentinel/blob/master/Detections%5CASimAuthentication%5CimAuthSigninsMultipleCountries.yaml', query="let timeframe = ago(3h);\nlet threshold = 2;\nimAuthentication\n| where TimeGenerated > timeframe\n| where EventType=='Logon' and EventResult=='Success'\n| where isnotempty(SrcGeoCountry)\n| summarize StartTime = min(TimeGenerated), EndTime = max(TimeGenerated), Vendors=make_set(EventVendor), Products=make_set(EventProduct)\n  , NumOfCountries = dcount(SrcGeoCountry)\n  by TargetUserId, TargetUsername, TargetUserType\n| where NumOfCountries >= threshold\n| extend timestamp = StartTime, AccountCustomEntity = TargetUsername\n", source_type='text', source_index=0, repo_name='Azure/Azure-Sentinel', query_name='User login from different countries within 3 hours (Uses Authentication Normalization)', context=None, attributes={'description': "'This query searches for successful user logins from different countries within 3 hours.\n To use this

,source_path,query,source_type,source_index,repo_name,query_name,context,attributes,kql_properties,query_id,query_hash,query_version
0,https://github.com/Azure/Azure-Sentinel/blob/m...,// You can leave out Anomalies that are alread...,text,0,Azure/Azure-Sentinel,Unusual Anomaly,None,{'description': ''Anomaly Rules generate event...,"{'functioncalls': ['ago'], 'joins': {' leftant...",72515608-2800-439c-9f39-43743f9a76e2,a81433962b6dd7226deb5f1e7c8761af9ab6ef9d6217f0...,0
1,https://github.com/Azure/Azure-Sentinel/blob/m...,let failureCountThreshold = 10;\nlet successCo...,text,0,Azure/Azure-Sentinel,Brute force attack against user credentials (U...,None,{'description': ''Identifies evidence of brute...,"{'functioncalls': ['min', 'max', 'make_set', '...",829a30db-2608-4784-a29d-c2e6dd058db6,7983ad26e75cb7525c70235533555ba8047164e6cfd74b...,0
2,https://github.com/Azure/Azure-Sentinel/blob/m...,let FailureThreshold = 15;\nimAuthentication\n...,text,0,Azure/Azure-Sentinel,Potential Password Spray Attack (Uses Authenti...,None,{'description': ''This query searches for fail...,"{'functioncalls': ['dcount', 'make_set', 'bin'...",86f97790-8c45-40b5-b76e-c2410a517b15,cc33ae898a65f31fd97da32adc2d389cac996905b90cd3...,0


## Get lists of attributes for filter controls

In [8]:
# 
ds.get_filter_lists()

# show approx items
print("indexes:")
print([f"{filter}: items: {len(vals)}" for filter, vals in ds.get_filter_lists().items()])
print("\nValues for 'operators' index")
print(ds.get_filter_lists()["operators"])

indexes:
['tactics: items: 35', 'techniques: items: 192', 'tables: items: 980', 'operators: items: 29', 'functioncalls: items: 203', 'joins: items: 0']

Values for 'operators' index
['as', 'count', 'distinct', 'evaluate', 'extend', 'filter', 'invoke', 'limit', 'make-series', 'mv-apply', 'mv-expand', 'mvexpand', 'order', 'parse', 'parse-where', 'partition', 'project', 'project-away', 'project-rename', 'project-reorder', 'render', 'serialize', 'sort', 'summarize', 'take', 'top', 'top-nested', 'union', 'where']


## Queries

In [9]:
# queries
# You can query on any field with "field=value"
# Simple query must be exact match
result = ds.find_queries(
    query_name="User login from different countries within 3 hours (Uses Authentication Normalization)"
)
print(result.shape)
display(result.head(3))

(1, 11)


,source_path,query,source_type,source_index,repo_name,query_name,context,attributes,kql_properties,query_hash,query_version
query_id,,,,,,,,,,,
9e569474-e0e8-4281-8a4d-7b14bba0b681,https://github.com/Azure/Azure-Sentinel/blob/m...,let timeframe = ago(3h);\nlet threshold = 2;\n...,text,0,Azure/Azure-Sentinel,User login from different countries within 3 h...,None,{'description': ''This query searches for succ...,"{'functioncalls': ['ago', 'isnotempty', 'min',...",47f3cf5bb4056439bb8e18ca4624c0ec2e8043ad62fb46...,0


In [10]:
result = ds.find_queries(query_name={"contains": "user"})
print(result.shape)
display(result.head(3))


(389, 11)


,source_path,query,source_type,source_index,repo_name,query_name,context,attributes,kql_properties,query_hash,query_version
query_id,,,,,,,,,,,
829a30db-2608-4784-a29d-c2e6dd058db6,https://github.com/Azure/Azure-Sentinel/blob/m...,let failureCountThreshold = 10;\nlet successCo...,text,0,Azure/Azure-Sentinel,Brute force attack against user credentials (U...,None,{'description': ''Identifies evidence of brute...,"{'functioncalls': ['min', 'max', 'make_set', '...",7983ad26e75cb7525c70235533555ba8047164e6cfd74b...,0
9e569474-e0e8-4281-8a4d-7b14bba0b681,https://github.com/Azure/Azure-Sentinel/blob/m...,let timeframe = ago(3h);\nlet threshold = 2;\n...,text,0,Azure/Azure-Sentinel,User login from different countries within 3 h...,None,{'description': ''This query searches for succ...,"{'functioncalls': ['ago', 'isnotempty', 'min',...",47f3cf5bb4056439bb8e18ca4624c0ec2e8043ad62fb46...,0
99259dd8-acbf-41a6-b304-4325526276f2,https://github.com/Azure/Azure-Sentinel/blob/m...,let known_users = (AuditLogs\n | where TimeGe...,text,0,Azure/Azure-Sentinel,Conditional Access Policy Modified by New User,None,{'description': ''Detects a Conditional Access...,"{'functioncalls': ['ago', 'tostring', 'parse_j...",246e62532d502dc18e3c859dc48a71264fca9091c88231...,0


In [11]:

# multiple conditions are ANDed - i.e. all must be true
result = ds.find_queries(
    query_name={"contains": "user"},
    repo_name={"contains": "reprise99"}
)
print(result.shape)
display(result.head(3))

(68, 11)


,source_path,query,source_type,source_index,repo_name,query_name,context,attributes,kql_properties,query_hash,query_version
query_id,,,,,,,,,,,
539ef59d-001b-4409-a4ac-aa37caaac682,https://github.com/reprise99/Sentinel-Queries/...,//Detects when unconstrained kerberos delegati...,text,0,reprise99/Sentinel-Queries,SecurityEvent-UnconstrainedDelegationtoUser,None,{},"{'functioncalls': [], 'joins': {}, 'operators'...",4bc453f54e846116d50edd8391b46c405d619900c5c3b5...,0
48099407-8a8c-4927-8027-a20492f871ab,https://github.com/reprise99/Sentinel-Queries/...,//When a user holding a privileged role trigge...,text,0,reprise99/Sentinel-Queries,Audit-EventsbyRiskyPrivilegedUser,None,{},"{'functioncalls': ['ago', 'arg_max', 'isnotemp...",bf294f42b09368f5695b4cb3c48a2e4a281f9160dd9a7e...,0
a2e159e6-3819-4045-b760-9236c18b5154,https://github.com/reprise99/Sentinel-Queries/...,//Find users who have failed 3 or more times t...,text,0,reprise99/Sentinel-Queries,Audit-FindUsersFailingNewPasswordSSPR,None,{},"{'functioncalls': ['tostring', 'parse_json', '...",531e6fd0d3b41f2fb826c9f0765d3f40241fa874a74086...,0


In [13]:
# Find all queries that have "user" in the name
# and use either SigninLogs or AuditLogs
result = ds.find_queries(
    query_name={"contains": "user"},
    tables=["SecurityAlert"] # the list values are OR'd - so will return UNION of matches
)
print(result.shape)
display(result.head(3))

(9, 11)


,source_path,query,source_type,source_index,repo_name,query_name,context,attributes,kql_properties,query_hash,query_version
query_id,,,,,,,,,,,
0d8ad554-e6db-4080-8835-2c8738cad27b,https://github.com/Azure/Azure-Sentinel/blob/m...,// Set the lookback to determine if user has c...,text,0,Azure/Azure-Sentinel,Azure DevOps Pipeline modified by a new user.,None,{'description': ''There are several potential ...,"{'functioncalls': ['ago', 'strcat', 'tolower',...",3ba73c49ac0a16ee6bd2af956146b80803ff322e87e650...,0
3af3f86e-e3a2-4c62-9da4-8433219b6efd,https://github.com/Azure/Azure-Sentinel/blob/m...,let Alerts = SecurityAlert\n| where AlertName ...,text,0,Azure/Azure-Sentinel,Mass Download & copy to USB device by single user,None,{'description': ''This query looks for any mas...,"{'functioncalls': ['parse_json', 'tostring', '...",697fbd3e51e58c50f6696962f59f9ee1a55fe7dd832630...,0
83ad1ff3-04ee-4406-8877-832eed30c03f,https://github.com/Azure/Azure-Sentinel/blob/m...,let AlertLinks = SecurityAlert\n| summarize hi...,text,0,Azure/Azure-Sentinel,Insider Risk_High User Security Alert Correlat...,None,{'description': ''This alert joins SecurityAle...,"{'functioncalls': ['arg_max', 'count', 'todyna...",8325412d12fad27c162597020aa9fd5a1a811a4112a955...,0


In [15]:
# Bad property name will throw exception
display(
    ds.find_queries(
        query_name={"contains": "user"},
        table=["SigninLogs", "AuditLogs"], # the list values are OR'd - so will return UNION
        operator=["mv-expand", "distinct"] # the list values are OR'd - so will return UNION
    ).shape
)

ValueError: Unknown attribute name {arg_name

In [18]:
# multiple filters
result = ds.find_queries(
    query_name={"contains": "user"},
    tables=["SigninLogs", "AuditLogs"], # the list values are OR'd - so will return UNION
    operators=["mv-expand", "distinct"] # the list values are OR'd - so will return UNION
)
print(result.shape)
display(result.head(3))

(27, 11)


,source_path,query,source_type,source_index,repo_name,query_name,context,attributes,kql_properties,query_hash,query_version
query_id,,,,,,,,,,,
5688beda-11f4-41ce-bdeb-024d05ccaee2,https://github.com/Azure/Azure-Sentinel/blob/m...,"AuditLogs\n | where OperationName has ""Consen...",text,0,Azure/Azure-Sentinel,End-user consent stopped due to risk-based con...,None,{'description': ''Detects a user's consent to ...,"{'functioncalls': ['tostring', 'parse_json', '...",e404742d11c204305d69d24a82e908c84103d20686a558...,0
74d08c7b-3630-4a28-8fba-9cd0460ea75c,https://github.com/Azure/Azure-Sentinel/blob/m...,"// Administrative roles to look for, default i...",text,0,Azure/Azure-Sentinel,New External User Granted Admin,None,{'description': ''This query will detect insta...,"{'functioncalls': ['tostring', 'isnotempty', '...",dff2d92ed33d9fe6e4f517e19935aeafd69f1ca6a9794e...,0
cb3aa662-6fbc-46ef-9776-b3266338a725,https://github.com/Azure/Azure-Sentinel/blob/m...,let lookback = 1d;\nAuditLogs \n| where TimeGe...,text,0,Azure/Azure-Sentinel,Suspicious linking of existing user to externa...,None,{'description': '' This query will detect when...,"{'functioncalls': ['ago', 'tostring', 'parse_j...",8ee32ee5daacf323516f5e178ffbbdb2510c39a67439e7...,0


In [19]:
# Use debug=True to get more insight into which item is matching what.
result = ds.find_queries(
    query_name={"contains": "user"},
    tables=["SigninLogs", "AuditLogs"], # the list values are OR'd - so will return UNION
    operators=["mv-expand", "distinct"], # the list values are OR'd - so will return UNION
    debug=True,
)
print(result.shape)
display(result.head(3))

{'contains': 'user'} False    2517
True      389
Name: query_name, dtype: int64
202
197
['SigninLogs', 'AuditLogs'] False    2814
True       92
Name: query_name, dtype: int64
230
150
['mv-expand', 'distinct'] False    2879
True       27
Name: query_name, dtype: int64
final criteria: False    2879
True       27
Name: query_name, dtype: int64
(27, 11)


,source_path,query,source_type,source_index,repo_name,query_name,context,attributes,kql_properties,query_hash,query_version
query_id,,,,,,,,,,,
5688beda-11f4-41ce-bdeb-024d05ccaee2,https://github.com/Azure/Azure-Sentinel/blob/m...,"AuditLogs\n | where OperationName has ""Consen...",text,0,Azure/Azure-Sentinel,End-user consent stopped due to risk-based con...,None,{'description': ''Detects a user's consent to ...,"{'functioncalls': ['tostring', 'parse_json', '...",e404742d11c204305d69d24a82e908c84103d20686a558...,0
74d08c7b-3630-4a28-8fba-9cd0460ea75c,https://github.com/Azure/Azure-Sentinel/blob/m...,"// Administrative roles to look for, default i...",text,0,Azure/Azure-Sentinel,New External User Granted Admin,None,{'description': ''This query will detect insta...,"{'functioncalls': ['tostring', 'isnotempty', '...",dff2d92ed33d9fe6e4f517e19935aeafd69f1ca6a9794e...,0
cb3aa662-6fbc-46ef-9776-b3266338a725,https://github.com/Azure/Azure-Sentinel/blob/m...,let lookback = 1d;\nAuditLogs \n| where TimeGe...,text,0,Azure/Azure-Sentinel,Suspicious linking of existing user to externa...,None,{'description': '' This query will detect when...,"{'functioncalls': ['ago', 'tostring', 'parse_j...",8ee32ee5daacf323516f5e178ffbbdb2510c39a67439e7...,0
